# OrdinalEncoder
- Se existe uma relação entre os rótulos de dados, podemos colocá-los em uma ordem e usar números para representar cada um dos rótulos
    - Ex: ruim / médio / bom pode ser representado como 0 / 1 / 2
- Independente do número de rótulos, é possível fazer a transformação **resultando em 1 única coluna**
- Vamos considerar a base de HR disponível no Kaggle:
    - https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists
    - [Essa base está disponível no drive](https://drive.google.com/file/d/1EdQSEnNFM8ufrW3831EItxjSaRhf0Vw9/view?usp=share_link)

In [1]:
# Importando o pandas
import pandas as pd

In [2]:
# Importando a base
base = pd.read_csv('aug_test.csv')

In [3]:
# Visualizando a base
base.head(2)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98


In [4]:
# E as informações dessa base
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             2129 non-null   int64  
 1   city                    2129 non-null   object 
 2   city_development_index  2129 non-null   float64
 3   gender                  1621 non-null   object 
 4   relevent_experience     2129 non-null   object 
 5   enrolled_university     2098 non-null   object 
 6   education_level         2077 non-null   object 
 7   major_discipline        1817 non-null   object 
 8   experience              2124 non-null   object 
 9   company_size            1507 non-null   object 
 10  company_type            1495 non-null   object 
 11  last_new_job            2089 non-null   object 
 12  training_hours          2129 non-null   int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 216.4+ KB


- Podemos verificar a coluna **education_level** para tratar o nível de educação do candidato

In [5]:
# Verificando a coluna education_level
base.education_level.value_counts()

Graduate          1269
Masters            496
High School        222
Phd                 54
Primary School      36
Name: education_level, dtype: int64

- Sabemos que existe uma relação entre o nível de educação
    - Por exemplo, começamos no ensino primário, depois vamos para a high school, graduate, masters e então phd
- Podemos começar transformando essas informações em colunas utilizando o One-Hot Encoding

In [6]:
# Transformando em colunas
pd.get_dummies(base.education_level)

,Graduate,High School,Masters,Phd,Primary School
0,1,0,0,0,0
1,1,0,0,0,0
2,0,1,0,0,0
3,0,0,1,0,0
4,1,0,0,0,0
...,...,...,...,...,...
2124,1,0,0,0,0
2125,0,0,1,0,0
2126,0,0,0,0,1
2127,0,1,0,0,0


- Só que nesse caso estamos criando 5 novas colunas no lugar de 1, então isso aumenta o tamanho da nossa base
- **Outra opção seria utilizar valores relacionados ao nível educacional do candidato (ex: 0 para primário, 1 para ensino médio (high school), 2 para graduação, etc)**

<a id="idOrdinalEncoder"></a>
- **Para isso, vamos utilizar o OrdinalEncoder**:
    - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder

In [7]:
# Importando
from sklearn.preprocessing import OrdinalEncoder

In [47]:
# Criando o objeto 
oe = OrdinalEncoder(categories=[['Primary School','High School','Graduate','Masters','Phd']]
                    ,handle_unknown='use_encoded_value',unknown_value=-1
                   )

In [48]:
# Fazendo o fit com os dados
oe = oe.fit(base[['education_level']])

In [49]:
# Transformando os dados
oe.transform(base[['education_level']])

array([[2.],
       [2.],
       [1.],
       ...,
       [0.],
       [1.],
       [3.]])

In [50]:
# Criando um DF
oe_df = pd.DataFrame(oe.transform(base[['education_level']]),columns=['education_level_oe'])
oe_df

,education_level_oe
0,2.0
1,2.0
2,1.0
3,3.0
4,2.0
...,...
2124,2.0
2125,3.0
2126,0.0
2127,1.0


In [51]:
# Eliminando a coluna education_level_oe
base = base.drop('education_level_oe',axis=1)

In [52]:
# Unindo com o dataset original
base = pd.concat([base,oe_df],axis=1)

In [53]:
# Visualizando a nova base
base.head(2)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,education_level_oe
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21,2.0
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98,2.0


In [54]:
# Considerando apenas as colunas de nível educacional
base[['education_level','education_level_oe']].value_counts()

education_level  education_level_oe
Graduate         2.0                   1269
Masters          3.0                    496
High School      1.0                    222
Phd              4.0                     54
Primary School   0.0                     36
dtype: int64

- Observe que o encoder que fizemos não está relacionando o nível correto da educação do candidato, pois ele está considerando um candidato com High School como 1, com ensino primário como 4 e graduado como 0
- Podemos [voltar](#idOrdinalEncoder) no OrdinalEncoder e definir `categories`

- Obs: ao usarmos o `categories`, vamos precisar informar para o OrdinalEncoder o que ele deve fazer com os valores vazios. Caso contrário, vamos ter um erro informando:<br>
`ValueError: Found unknown categories [nan] in column ___ during fit`

In [46]:
# Podemos também filtrar os valores com education_level vazios
base.loc[base.education_level.isnull(),'education_level_oe'].value_counts()

-1.0    52
Name: education_level_oe, dtype: int64

- Também podemos passar o `dtype` como parâmetro do próprio OrdinalEncoder para determinar o tipo dos dados

- Para transformar mais de 1 coluna, basta passar essas informações nas categorias

In [ ]:
# Adicionando a coluna de enrolled_university na análise

In [ ]:
# Definindo a ordem dos rótulos das duas colunas

In [ ]:
# E usando o OrdinalEncoder

In [ ]:
# Fazendo o fit com os dados

In [ ]:
# Transformando os dados